In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

ModuleNotFoundError: No module named 'pandas'

In [2]:
df = pd.read_csv('datasett/meow_cleaned.csv')
breedLabelsDf = pd.read_csv('datasett/BreedLabels.csv')
colorLabelsDf = pd.read_csv('datasett/ColorLabels.csv')
stateLabelsDf = pd.read_csv('datasett/StateLabels.csv')

# Convert non number columns to number by mapping it to labels df

In [27]:
df_cleaned = df.copy()
df_cleaned.drop(['Unnamed: 0', 'Name', 'PetID', 'AdoptionSpeedBool'], axis=1, inplace=True)
df_cleaned['Type'] = df_cleaned['Type'].apply(lambda x: 1 if x == 'Dog' else 2)

breed_mapping = breedLabelsDf.set_index('BreedName')['BreedID'].to_dict()
df_cleaned['Breed1'] = df_cleaned['Breed1'].apply(lambda x: breed_mapping.get(x, None))
df_cleaned['Breed2'] = df_cleaned['Breed2'].apply(lambda x: breed_mapping.get(x, None))

gender_mapping = {'Male': 1, 'Female': 2, 'Mixed': 3}
df_cleaned['Gender'] = df_cleaned['Gender'].map(gender_mapping)

color_mapping = colorLabelsDf.set_index('ColorName')['ColorID'].to_dict()
df_cleaned['Color1'] = df_cleaned['Color1'].apply(lambda x: color_mapping.get(x, None))
df_cleaned['Color2'] = df_cleaned['Color2'].apply(lambda x: color_mapping.get(x, None))
df_cleaned['Color3'] = df_cleaned['Color3'].apply(lambda x: color_mapping.get(x, None))

maturity_size_mapping = {'Small': 1, 'Medium': 2, 'Large': 3, 'Extra Large': 4}
df_cleaned['MaturitySize'] = df_cleaned['MaturitySize'].map(maturity_size_mapping)

fur_length_mapping = {'Short': 1, 'Medium': 2, 'Long': 3}
df_cleaned['FurLength'] = df_cleaned['FurLength'].map(fur_length_mapping)

vaccinated_mapping = {'Yes': 1, 'No': 2}
df_cleaned['Vaccinated'] = df_cleaned['Vaccinated'].map(vaccinated_mapping).fillna(0)
df_cleaned['Dewormed'] = df_cleaned['Dewormed'].map(vaccinated_mapping).fillna(0)
df_cleaned['Sterilized'] = df_cleaned['Sterilized'].map(vaccinated_mapping).fillna(0)

health_mapping = {'Healthy': 1, 'Minor Injury': 2, 'Serious Injury': 3}
df_cleaned['Health'] = df_cleaned['Health'].map(health_mapping)

state_mapping = stateLabelsDf.set_index('StateName')['StateID'].to_dict()
df_cleaned['State'] = df_cleaned['State'].map(state_mapping)

df_cleaned.head()

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,State,Quantity,Fee,VideoAmt,PhotoAmt,AdoptionSpeed
0,2,3,299,299,1,1,7,7,1,1,2.0,2.0,2.0,1,41326,1,100,0,1.0,2
1,2,1,265,265,1,1,2,2,2,2,0.0,0.0,0.0,1,41401,1,0,0,2.0,0
2,1,1,307,307,1,2,7,7,2,2,1.0,1.0,2.0,1,41326,1,0,0,7.0,3
3,1,4,307,307,2,1,2,2,2,1,1.0,1.0,2.0,1,41401,1,150,0,8.0,2
4,1,1,307,307,1,1,1,1,2,1,2.0,2.0,2.0,1,41326,1,0,0,3.0,2


# One hot encode the states (made it marginally better but maybe still useful)

In [29]:
df_cleaned_one_hot_encode_state = df_cleaned.copy()
df_cleaned_one_hot_encode_state = pd.get_dummies(df_cleaned_one_hot_encode_state, columns=['State'], drop_first=True).astype(int)
df_cleaned_one_hot_encode_state.head()

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,...,State_41330,State_41332,State_41335,State_41336,State_41342,State_41345,State_41361,State_41367,State_41401,State_41415
0,2,3,299,299,1,1,7,7,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,1,265,265,1,1,2,2,2,2,...,0,0,0,0,0,0,0,0,1,0
2,1,1,307,307,1,2,7,7,2,2,...,0,0,0,0,0,0,0,0,0,0
3,1,4,307,307,2,1,2,2,2,1,...,0,0,0,0,0,0,0,0,1,0
4,1,1,307,307,1,1,1,1,2,1,...,0,0,0,0,0,0,0,0,0,0


# Splitting to train and test set

In [33]:
df_cleaned_x = df_cleaned.copy()
X = df_cleaned_x.drop('AdoptionSpeed', axis=1)
y = df_cleaned['AdoptionSpeed']

# Splitting data into training and test sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [34]:
model = DecisionTreeClassifier(random_state=42, max_depth=5)
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5, random_state=42)

In [35]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)

# Classification report
cr = classification_report(y_test, y_pred)
print('Classification Report:')
print(cr)

Accuracy: 0.3882
Confusion Matrix:
[[  1  40  11   0  27]
 [  0 263 196  15 146]
 [  0 255 305  27 224]
 [  0 149 241  50 204]
 [  0 122 151  23 543]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.01      0.03        79
           1       0.32      0.42      0.36       620
           2       0.34      0.38      0.36       811
           3       0.43      0.08      0.13       644
           4       0.47      0.65      0.55       839

    accuracy                           0.39      2993
   macro avg       0.51      0.31      0.28      2993
weighted avg       0.41      0.39      0.35      2993

